In [1]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Step 1. Load Dataset
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=False)

# Step 2. Make Dataset Iterable
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
# Step 3. Create Model Class
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, int(input_dim/2))
        self.linear2 = torch.nn.Linear(int(input_dim/2), int(input_dim/4))
        self.linear3 = torch.nn.Linear(int(input_dim/4), output_dim)

    def forward(self, x):
        outputs = self.linear1(x)
        outputs = self.linear2(outputs)
        outputs = self.linear3(outputs)
        
        return outputs

In [17]:
n_iters = 3000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 784 # 28 * 28
output_dim = 10 # class가 10개이므로
# [test] 만일 MSE을 LOSS 함수로 쓴다면???
# output_dim = 1
lr_rate = 0.001

# Step 4. Instantiate Model Class
model = LogisticRegression(input_dim, output_dim)
# Step 5. Instantiate Loss Class
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy
# [test] 만일 MSE을 LOSS 함수로 쓴다면???
# criterion = torch.nn.MSELoss()
# Step 6. Instantiate Optimizer Class
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [18]:
# Step 7. Train Model
iter = 0
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        # [test] 만일 MSE을 LOSS 함수로 쓴다면???
        # labels = labels.type(torch.FloatTensor)
        # outputs = outputs.type(torch.FloatTensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter+=1
        if iter%500==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28*28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
            accuracy = 100 * correct/total
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.206150770187378. Accuracy: 53.040000915527344.
Iteration: 1000. Loss: 2.0966529846191406. Accuracy: 66.5999984741211.
Iteration: 1500. Loss: 1.916388750076294. Accuracy: 71.18000030517578.
Iteration: 2000. Loss: 1.782907247543335. Accuracy: 72.47000122070312.
Iteration: 2500. Loss: 1.546745777130127. Accuracy: 73.87000274658203.
Iteration: 3000. Loss: 1.4253721237182617. Accuracy: 75.62000274658203.
